In [ ]:
%autosave 60

In [ ]:
import torch.nn as nn
import torch
import argparse
import os

import numpy as np

import copy
import torch



from tqdm.notebook import tqdm
from torch.autograd import Variable
from torch.autograd import grad as torch_grad
import matplotlib.pyplot as plt
# from torch.utils.tensorboard import writer, SummaryWriter
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch.utils.data import RandomSampler
from math import pi
import pandas as pd
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import roc_auc_score,average_precision_score,recall_score,precision_score

from torch.utils.data import WeightedRandomSampler
from torch.utils.data import Subset

# from datasets.datasets import Sines
# from models.wgangp import Generator, Critic



# Articles
+ https://arxiv.org/pdf/1803.01271.pdf -TCN
+ https://arxiv.org/pdf/2005.01181.pdf - GAN Forecasting
+ https://papers.nips.cc/paper/2019/file/c9efe5f26cd17ba6216bbe2a7d26d490-Paper.pdf - TimeGAN (откуда у них реконстрашн лосс?)
+ https://habr.com/ru/post/352794/ gan explained
+ https://habr.com/ru/post/447494/
+ https://www.analyticsvidhya.com/blog/2017/06/introductory-generative-adversarial-networks-gans/ - обучение GAN
+ https://id-lab.ru/posts/developers/sovety-po-obucheniyu-stabilnyh-generativno-sostyazatelnyh-setej-gan/ обучение GAN

In [ ]:
import pyreadr

normal_train = pyreadr.read_r('data/TEP_FaultFree_Training.RData') # also works for Rds
normal_test = pyreadr.read_r('data/TEP_FaultFree_Testing.RData')
faulty_train = pyreadr.read_r('data/TEP_Faulty_Training.RData')
faulty_test = pyreadr.read_r('data/TEP_Faulty_Testing.RData')




In [ ]:
df_n_tr = normal_train['fault_free_training']
df_n_te = normal_test['fault_free_testing']
df_a_tr = faulty_train['faulty_training']
df_a_te = faulty_test['faulty_testing']

In [ ]:
grouped = df_a_tr.groupby(['faultNumber','simulationRun'])

In [ ]:
def get_data(df_n_tr,df_n_te,df_a_tr,df_a_te,seed=1,fault_number=1):
    
    train_normal=df_n_tr[df_n_tr['simulationRun']==seed]
    test_normal=df_n_te[df_n_te['simulationRun']==seed]
    
    train_abnormal=df_a_tr[(df_a_tr['faultNumber']==fault_number)&(df_a_tr['simulationRun']==seed)]
    test_abnormal=df_a_te[(df_a_te['faultNumber']==fault_number)&(df_a_te['simulationRun']==seed)]
    
    return train_normal,test_normal,train_abnormal,test_abnormal
    

In [ ]:
# train_normal,test_normal,train_abnormal,test_abnormal = get_data(df_n_tr,df_n_te,df_a_tr,df_a_te,seed=1,fault_number=1)

In [ ]:
features=['xmeas_1', 'xmeas_2', 'xmeas_3', 'xmeas_4', 'xmeas_5', 'xmeas_6',
       'xmeas_7', 'xmeas_8', 'xmeas_9', 'xmeas_10', 'xmeas_11', 'xmeas_12',
       'xmeas_13', 'xmeas_14', 'xmeas_15', 'xmeas_16', 'xmeas_17', 'xmeas_18',
       'xmeas_19', 'xmeas_20', 'xmeas_21', 'xmeas_22', 'xmeas_23', 'xmeas_24',
       'xmeas_25', 'xmeas_26', 'xmeas_27', 'xmeas_28', 'xmeas_29', 'xmeas_30',
       'xmeas_31', 'xmeas_32', 'xmeas_33', 'xmeas_34', 'xmeas_35', 'xmeas_36',
       'xmeas_37', 'xmeas_38', 'xmeas_39', 'xmeas_40', 'xmeas_41', 'xmv_1',
       'xmv_2', 'xmv_3', 'xmv_4', 'xmv_5', 'xmv_6', 'xmv_7', 'xmv_8', 'xmv_9',
       'xmv_10', 'xmv_11']

In [ ]:
len(features)

In [ ]:
class MyPrettyDataset(Dataset):
    """The most sexy dataset in the world"""

    def __init__(self, data,sample_len=15):
        
        
        self.features=['xmeas_1', 'xmeas_2', 'xmeas_3', 'xmeas_4', 'xmeas_5', 'xmeas_6',
       'xmeas_7', 'xmeas_8', 'xmeas_9', 'xmeas_10', 'xmeas_11', 'xmeas_12',
       'xmeas_13', 'xmeas_14', 'xmeas_15', 'xmeas_16', 'xmeas_17', 'xmeas_18',
       'xmeas_19', 'xmeas_20', 'xmeas_21', 'xmeas_22', 'xmeas_23', 'xmeas_24',
       'xmeas_25', 'xmeas_26', 'xmeas_27', 'xmeas_28', 'xmeas_29', 'xmeas_30',
       'xmeas_31', 'xmeas_32', 'xmeas_33', 'xmeas_34', 'xmeas_35', 'xmeas_36',
       'xmeas_37', 'xmeas_38', 'xmeas_39', 'xmeas_40', 'xmeas_41', 'xmv_1',
       'xmv_2', 'xmv_3', 'xmv_4', 'xmv_5', 'xmv_6', 'xmv_7', 'xmv_8', 'xmv_9',
       'xmv_10', 'xmv_11']
        self.data = data
        self.group_size=500
        self.grouped = self.data.groupby(['faultNumber','simulationRun'])
        self.sample_len=sample_len
        self.max_group_index = self.group_size - self.sample_len +1
        self.groups_count = len(self.grouped)
        self.group_keys=list(self.grouped.groups.keys())
        
        self.groups = [self.grouped.get_group(key) for key in self.group_keys]
        

    def __len__(self):
        return len(self.grouped)*(self.max_group_index)
    
    def __getitem__(self, idx):
        group_index = idx//(self.max_group_index)
        idx = idx%(self.max_group_index)
        group = self.groups[group_index]#self.grouped.get_group(self.group_keys[group_index])
        
        fault_numbers = group['faultNumber'].unique()

        is_anomal = False
        if len(fault_numbers)>1:
            raise Exception
        elif len(fault_numbers)==1:
            fault = fault_numbers[0]
            if fault >=1:
                is_anomal=1
            else:
                is_anomal=0
        
        return group.iloc[idx:idx+self.sample_len][self.features].values.astype(float),is_anomal
            
        

        

In [ ]:
# conc = pd.concat([df_n_tr,df_a_tr[df_a_tr['faultNumber'].isin([1,2,3])]])
conc = pd.concat([df_n_tr,df_a_tr])

In [ ]:


scaler = MinMaxScaler()
scaler.fit(conc[features].values)
values = scaler.transform(conc[features].values)
conc_scaled = conc.copy()
for i, col in enumerate(features):
    conc_scaled[col] = values[:, i]
conc_scaled.head()

In [ ]:
class MyPrettyDataset(Dataset):
    """The most sexy dataset in the world"""

    def __init__(self, sample_len=15):
        
        
       
        self.sample_len=sample_len
        
    def __len__(self):
        return 10_000#len(self.grouped)*(self.max_group_index)
    
    def __getitem__(self, idx):
        t = torch.tensor(0)
        t=t.new_full((self.sample_len,52),idx%2)
        
        return t,idx%2
            
        

        

In [ ]:
class Generator(nn.Module):
    def __init__(self,noise_dim=8,target_len=450,hidden_dim=16,label_emb_dim=8,input_dim=8,dropout=0.2):
        super().__init__()
        
        self.label_embedder = nn.Embedding(2,label_emb_dim)
        self.predictor = nn.Sequential(
                nn.Linear(hidden_dim,128),
                nn.BatchNorm1d(128),
                nn.ReLU(),
                nn.Dropout(dropout),
            
                nn.Linear(128, 512),
                nn.BatchNorm1d(512),
                nn.ReLU(),
                nn.Dropout(dropout),
            
                nn.Linear(512, 256),
                nn.BatchNorm1d(256),
                nn.ReLU(),
            
                nn.Linear(256, 52)
        )
        self.label_noise_embedder = nn.Linear(label_emb_dim,52)#noise_dim,52)
        
        self.generator = nn.LSTM(52,hidden_dim,batch_first=True)
        
        self.activation = nn.LeakyReLU()
        self.target_len = target_len
    
        
        

    def forward(self,noise,labels):
        
        
        label_embedding = self.label_embedder(labels).squeeze(1)
        
#         input = self.activation(self.label_noise_embedder(torch.cat([label_embedding,noise],dim=1))).unsqueeze(1)
        input = label_embedding+noise
        input = self.label_noise_embedder(input).unsqueeze(1)
#         print(input.shape)
    
        hidden=None
        for i in range(self.target_len):
            output,hidden = self.generator(input[:,i,:].unsqueeze(1),hidden)
            output =self.activation(self.predictor(output[:,-1,:]))
            # print(input.shape,output.shape)
            input = torch.cat([input,output.unsqueeze(1)],dim=1)
            
        return input[:,1:,:]
            
        
       
    
    



In [ ]:
# class Generator(nn.Module):
#     def __init__(self,noise_dim,target_len=450,hidden_dim=16,label_emb_dim=3,input_dim=8,dropout=0.2):
#         super().__init__()
        
#         self.label_embedder = nn.Embedding(2,label_emb_dim)
#         self.predictor = nn.Sequential(
#                 nn.Linear(hidden_dim,8),
#                 nn.BatchNorm1d(8),
#                 nn.ReLU(),
#                 nn.Dropout(dropout),
            
#                 nn.Linear(8, 16),
#                 nn.BatchNorm1d(16),
#                 nn.ReLU(),
#                 nn.Dropout(dropout),
            
#                 nn.Linear(16, 32),
#                 nn.BatchNorm1d(32),
#                 nn.ReLU(),
            
#                 nn.Linear(32, 52)
#         )
#         self.label_noise_embedder = nn.Linear(label_emb_dim+noise_dim,52)
        
#         self.generator = nn.LSTM(52,hidden_dim,batch_first=True)
        
#         self.activation = nn.LeakyReLU()
#         self.target_len = target_len
    
        
        

#     def forward(self,noise,labels):
        
        
#         label_embedding = self.label_embedder(labels).squeeze(1)
        
#         input = self.activation(self.label_noise_embedder(torch.cat([label_embedding,noise],dim=1))).unsqueeze(1)
        
        
#         hidden=None
#         for i in range(self.target_len):
#             output,hidden = self.generator(input[:,i,:].unsqueeze(1),hidden)
#             output =self.activation(self.predictor(output[:,-1,:]))
#             # print(input.shape,output.shape)
#             input = torch.cat([input,output.unsqueeze(1)],dim=1)
            
#         return input[:,1:,:]
            
        
       
    
    



In [ ]:
class Discriminator(nn.Module):
    def __init__(self,detectors_count=52,hidden_dim=8,label_emb_dim=3,dropout=0.1):
        super().__init__()
        
        self.lstm = nn.LSTM(detectors_count,hidden_dim,batch_first=True,num_layers=2)
        self.label_embedder  = nn.Embedding(2,label_emb_dim)
        self.model = nn.Sequential(
                nn.Linear(hidden_dim+label_emb_dim, 512),
                nn.BatchNorm1d(512),
                nn.LeakyReLU(),
                nn.Dropout(dropout),
            
            
                nn.Linear(512, 128),
                nn.BatchNorm1d(128),
                nn.LeakyReLU(),
                nn.Dropout(dropout),
            
                nn.Linear(128, 8),
                nn.BatchNorm1d(8),
                nn.LeakyReLU(),
            
                
            
                nn.Linear(8, 2),
                nn.Softmax(dim=1)
        )
        
        self.sigmoid = nn.Sigmoid()

    def forward(self,input,labels):
        label_embedding = self.label_embedder(labels)
        output,hidden = self.lstm(input.float())
        x = output[:,-1,:]
        x = torch.cat([x,label_embedding],dim=1)
        x = self.model(x)
        return x
    


In [ ]:
NOISE_DIM=8
BATCH_SIZE=100
EPOCHS=10000
SAMPLE_LEN=10
PART_OF_SYNTHETIC=2 # поровну 

anomally_label=1
normal_label=0

DISCRIMINATOR_EPOCH=10


g = Generator(noise_dim=8,target_len=SAMPLE_LEN,hidden_dim=16,label_emb_dim=8,input_dim=8)
d = Discriminator()



g_opt = torch.optim.Adam(g.parameters(), lr = 0.001)

d_opt = torch.optim.Adam(d.parameters(), lr = 0.001) # 



g_loss_f = nn.BCELoss()
d_loss_f = nn.BCELoss()






In [ ]:
dataset = MyPrettyDataset(sample_len=SAMPLE_LEN)#conc_scaled,sample_len=450)

In [ ]:
data_subset = Subset(dataset,np.random.randint(len(dataset),size=10**3))

In [ ]:
dataloader = DataLoader(data_subset,batch_size=BATCH_SIZE,num_workers=0,shuffle=True,drop_last=True)

In [ ]:
def make_weights(dataset,weights_d={0:0.5,1:0.1}):
    weights=[]
    for x,y in tqdm(dataset):
        weights.append(weights_d[y])
    return weights

In [ ]:
weights = make_weights(data_subset)

In [ ]:
def log_gradients_in_model(model, logger, step):
    for tag, value in model.named_parameters():
        if value.grad is not None:
#             print(value.grad.cpu())
            logger.add_histogram(model._get_name()+'/'+tag + "/grad", value.grad.cpu(), step)

In [ ]:
def get_label_distribution(generator,label=0,noise_dim=8,num_samples=10000):
    generator.eval()
    noise = torch.randn((num_samples,noise_dim))
    if label==0:
        labels=torch.zeros((num_samples,1),dtype=torch.int32)
    elif label==1:
        labels=torch.ones((num_samples,1),dtype=torch.int32)
    output = generator(noise,labels)
    output = output[:,-1,0]
    return output.flatten()
    

In [ ]:
fixed_noise=torch.randn((real_data.shape[0],NOISE_DIM))
fixed_labels=anomally_labels

In [ ]:
def train_gen(batch_size,labels):
    
    noise = torch.rand(BATCH_SIZE, NOISE_DIM)
    gen_data = g(noise,labels)
    pred = d(gen_data,labels)

    target = torch.ones(BATCH_SIZE).long()
#     print(target.shape,pred.shape)
    loss = F.cross_entropy(pred, target)
    
    
    g_opt.zero_grad()
    loss.backward()
    g_opt.step()
    return loss.item()

In [ ]:
def train_dics(batch,labels):
    
    
    real_data = batch
    
    with torch.no_grad():
        noise = torch.rand(BATCH_SIZE, NOISE_DIM)
        gen_data = g(noise,labels)
    
    data = torch.cat([gen_data, real_data])
    
    pred = d(data,torch.cat([labels,labels]))

    target = torch.cat([torch.zeros(BATCH_SIZE), 
                        torch.ones(BATCH_SIZE)]).long()

    loss = F.cross_entropy(pred, target)
#     return pred,target
    d_opt.zero_grad()
    loss.backward()
    d_opt.step()
    return loss.item()

In [ ]:
writer = SummaryWriter()
for epoch in tqdm(range(EPOCHS),desc='Epoch'):
    d_sum_loss=0
    g_sum_loss = 0
    for real_data,anomally_labels in tqdm(dataloader,desc='Batch'):
        for _ in range(5):
            disc_loss = train_dics(real_data,anomally_labels)
        gen_loss = train_gen(BATCH_SIZE,anomally_labels)
#         print(disc_loss,gen_loss)
        d_sum_loss+=disc_loss
        g_sum_loss+=gen_loss
        
    # Inference generator
    # with torch.no_grad():
    #     g.eval()
    #     noise = torch.randn(size=(1,NOISE_DIM))
    #     idxs = [0]
    #     fake_data=g(data[idxs],label=anomally_labels[idxs],noise=noise)
    
    # log_gradients_in_model(g, writer, epoch)
    # log_gradients_in_model(d, writer, epoch)
    writer.add_histogram("Distribution generator output", g(noise=fixed_noise,labels=fixed_labels).flatten(), epoch)
    
    writer.add_histogram("Distribution discriminator output", d(g(noise=fixed_noise,labels=fixed_labels),labels=fixed_labels).flatten(), epoch)
    # writer.add_scalars('Components',
    #                    {f'Real component':data[0,-1,0].item(),
    #                     'Synthetic component':fake_data[0,-1,0].item()},
    #                    epoch)
    writer.add_scalar('Labels count',anomally_labels.sum(),epoch)
#     writer.add_scalar('Classification metrics/ROC AUC',roc_auc_score(train_anomally_labels.numpy(),d_output.numpy(),labels=None),epoch)
#     writer.add_scalar('Classification metrics/Recall ',recall_score(train_anomally_labels.numpy(),(d_output.numpy()>0.5)*1),epoch)
#     writer.add_scalar('Classification metrics/Precision ',precision_score(train_anomally_labels.numpy(),(d_output.numpy()>0.5)*1,zero_division=0),epoch)
    writer.add_scalar('Loss/generator loss',g_sum_loss/len(dataloader),epoch)
    writer.add_scalar('Loss/discriminator loss',d_sum_loss/len(dataloader),epoch)
        
        
        
        


In [ ]:
writer = SummaryWriter()
for epoch in tqdm(range(EPOCHS),desc='Epoch'):
    d_sum_loss=0
    g_sum_loss = 0
    for real_data,anomally_labels in tqdm(dataloader,desc='Batch'):
        d.train()
        g.train()
        
        # Optimize discriminator
        noise=torch.randn((real_data.shape[0],NOISE_DIM))
        
        with torch.no_grad():
            fake_data=g(noise=noise,labels=anomally_labels)
        
        
        data = torch.cat([real_data,fake_data],dim=0)
        train_anomally_labels = torch.cat([anomally_labels,anomally_labels],dim=0)
        
        is_generated_labels = torch.cat([torch.zeros(real_data.shape[0]),torch.ones(fake_data.shape[0])],dim=0).unsqueeze(1)
        inv_is_generated_labels = (is_generated_labels+1)%2
        for _ in range(DISCRIMINATOR_EPOCH):
        
            d_output = d(data,labels=train_anomally_labels)

            d_loss = d_loss_f(d_output,is_generated_labels) 
            d_sum_loss+=d_loss.item()
            d_opt.zero_grad()
            d_loss.backward()
            d_opt.step()
        
        
      
        # Optimize generator
        fake_data=g(labels=anomally_labels,noise=noise)
        # print(fake_data)
        
        
        data = torch.cat([real_data,fake_data],dim=0)
        train_anomally_labels = torch.cat([anomally_labels,anomally_labels],dim=0)
        
        is_generated_labels = torch.cat([torch.zeros(real_data.shape[0]),torch.ones(fake_data.shape[0])],dim=0).unsqueeze(1)
        inv_is_generated_labels = (is_generated_labels+1)%2
        
        
        d_output = d(data,labels=train_anomally_labels)
        
        g_loss = g_loss_f(d_output,inv_is_generated_labels)
        g_sum_loss+=g_loss.item()
        
        g_opt.zero_grad()
        g_loss.backward()
        g_opt.step()
        fake_data=g(labels=anomally_labels,noise=noise)
#         print(fake_data)
        
        
    # Inference discriminator
        
    with torch.no_grad():
        d.eval()
        d_output = d(data,labels=torch.zeros(data.shape[0],dtype=torch.int))
        
        
    # Inference generator
    # with torch.no_grad():
    #     g.eval()
    #     noise = torch.randn(size=(1,NOISE_DIM))
    #     idxs = [0]
    #     fake_data=g(data[idxs],label=anomally_labels[idxs],noise=noise)
    
    # log_gradients_in_model(g, writer, epoch)
    # log_gradients_in_model(d, writer, epoch)
    writer.add_histogram("Distribution generator output", g(noise=fixed_noise,labels=fixed_labels).flatten(), epoch)
    # writer.add_scalars('Components',
    #                    {f'Real component':data[0,-1,0].item(),
    #                     'Synthetic component':fake_data[0,-1,0].item()},
    #                    epoch)
    writer.add_scalar('Labels count',anomally_labels.sum(),epoch)
    writer.add_scalar('Classification metrics/ROC AUC',roc_auc_score(train_anomally_labels.numpy(),d_output.numpy(),labels=None),epoch)
    writer.add_scalar('Classification metrics/Recall ',recall_score(train_anomally_labels.numpy(),(d_output.numpy()>0.5)*1),epoch)
    writer.add_scalar('Classification metrics/Precision ',precision_score(train_anomally_labels.numpy(),(d_output.numpy()>0.5)*1,zero_division=0),epoch)
    writer.add_scalar('Loss/generator loss',g_sum_loss/len(dataloader),epoch)
    writer.add_scalar('Loss/discriminator loss',d_sum_loss/len(dataloader),epoch)
        
        
        
        
